In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import cross_val_score, KFold
from sklearn import (
    compose,
    linear_model,
    model_selection,
    preprocessing,
    feature_selection,
    svm
)
from sklearn import pipeline
from sklearn.inspection import permutation_importance

from tqdm import tqdm

import warnings
from os import environ

In [2]:
warnings.simplefilter("ignore")
environ["PYTHONWARNINGS"] = "ignore"

In [3]:
kf = model_selection.RepeatedKFold(n_repeats=5, n_splits=5, random_state=42)

In [4]:
df = pd.read_csv('logP.csv')
y = df.logP
XX = df.iloc[:, 3:]

FileNotFoundError: [Errno 2] No such file or directory: 'logP.csv'

In [ ]:
X = XX[XX.columns[XX.var() != 0]] 
feats = X.columns         # без колонок с нулевой var
X                       

,QED_score,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_priamide,fr_pyridine,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_tetrazole,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0.909,12.053455,12.053455,0.045556,0.045556,0.909253,281.355,262.203,281.141579,108,...,0,0,0,0,0,0,0,0,0,0
1,0.839,12.616874,12.616874,0.009514,-5.052156,0.839542,262.183,253.111,262.045293,98,...,0,0,0,0,0,0,0,0,0,0
2,0.828,4.217020,4.217020,1.056122,1.056122,0.828333,266.388,244.212,266.178299,104,...,0,0,0,0,0,0,0,0,0,0
3,0.726,11.239605,11.239605,0.364327,-1.508792,0.726080,318.372,300.228,318.125594,120,...,0,0,0,0,0,0,0,0,0,0
4,0.816,4.352931,4.352931,0.140811,0.140811,0.816043,214.312,196.168,214.146999,84,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,0.863,12.562081,12.562081,0.093167,-0.455663,0.863227,381.501,358.317,381.151098,140,...,0,0,1,0,0,0,0,0,0,0
240,0.737,12.114099,12.114099,0.072506,-0.211498,0.737878,321.376,302.224,321.136493,122,...,0,0,0,0,0,0,0,0,0,0
241,0.824,12.974979,12.974979,0.339313,-1.732014,0.824192,274.266,262.170,274.080536,102,...,0,0,0,0,0,0,0,0,0,0
242,0.750,12.347197,12.347197,0.125336,-0.724707,0.750035,321.312,308.208,321.088080,120,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
float_columns = X.select_dtypes(np.float64).columns         # 102
int_columns = X.select_dtypes(np.int64).columns             # 75

In [ ]:
prep = compose.make_column_transformer(
                (preprocessing.StandardScaler(), float_columns),
                (preprocessing.MinMaxScaler(feature_range=(0, 1)), int_columns),        # стандартизовали два типа колонок
                remainder = "passthrough"
            )

_X = prep.fit_transform(X)

X_new = pd.DataFrame(_X, columns = feats)

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(  X_new, y, test_size=0.2, random_state=42)

In [14]:

cross_val_score(linear_model.LassoCV(), X_new, y, cv = kf).mean()

0.43565111254082983

In [11]:
model = linear_model.LassoCV(cv = kf)
model.fit(X_new, y)

model.score(X_new, y)

0.6290354923155224

In [10]:
model = linear_model.LassoCV(cv = kf)
model.fit(X_train, y_train)

model.score(X_test, y_test)

0.3662873563912007

In [16]:
model2 = linear_model.RidgeCV(cv = kf)
model2.fit(X_train, y_train)

model2.score(X_test, y_test)

0.2291661529159229

## feature selection

In [18]:
model_rfecv = feature_selection.RFECV(estimator=linear_model.LassoCV(), min_features_to_select=5, cv=kf, n_jobs=-2)

model_rfecv.fit(X_new, y)